# **Initial Setup of Portfolio**

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

save_path = "/content/gdrive/MyDrive/XML Qns/"

Mounted at /content/gdrive


In [ ]:
!pip install selenium
from selenium import webdriver
!apt-get install chromium-driver
!pip install pyodbc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'chromium-chromedriver' instead of 'chromium-driver'
The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb
  udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 9 newly installed, 0 to remove and 48 not upgraded.
Need to get 29.0 MB of archives.
After this operation, 120 MB of additional disk

In [ ]:
import pandas as pd
import time
import re
import os
import pyodbc
import pytz
import numpy as np
import statistics
from math import *
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

In [ ]:
def web_driver():
  options = webdriver.ChromeOptions()
  options.add_argument("--verbose")
  options.add_argument("--no-sandbox")
  options.add_argument("--headless")
  options.add_argument("--disable-gpu")
  options.add_argument("--window-soze = 1920, 1200")
  options.add_argument("--disable-dev-shm-usage")
  driver = webdriver.Chrome(options = options)
  return driver

driver = web_driver()

def N(x):
  #'Cumulative distribution function for the standard normal distribution'
  return (1.0 + erf(x / sqrt(2.0))) / 2.0

def deltac(S, K, r, y, τ, σ):
  d1 = 1/(σ * np.sqrt(τ)) * (np.log(S * np.exp(-y * (τ)) / K) + (r - y + σ**2/2)*(τ) )
  return np.exp(-y * τ) * N(d1)

def c(S, K, r, y, τ, σ):
  d1 = 1/(σ * np.sqrt(τ)) * (np.log(S * np.exp(-y * (τ)) / K) + (r - y + σ**2/2)*(τ) )
  d2 = d1 - σ * np.sqrt(τ)
  return S * np.exp(-y * (τ)) * N(d1) - K * np.exp(-r * (τ)) * N(d2)

time_sleep = 0.7

In [ ]:
#stock to search
option = 'NVDA250117C00140000'
ticker = 'NVDA'
r = 0.0034 #risk free sg rate as of oct 2024

In [ ]:
url = "https://sg.finance.yahoo.com/"

# Open the webpage
driver.get(url)

# Wait for the page to load
time.sleep(0.1)

# get search button element
search = driver.find_element(By.XPATH, '//*[@id="header-quote-lookup"]/div/form/input')
# create action chain object
action = ActionChains(driver)
# click the item
action.click(on_element = search)
# perform the operation
action.perform()


#search ticker
search.send_keys(ticker)
search.send_keys(Keys.ENTER)
time.sleep(time_sleep)

S = float(driver.find_elements(By.XPATH, '/html/body/div[2]/main/section/section/section/article/section[1]/div[2]/div[1]/section/div/section[1]/div[1]/fin-streamer[1]')[0].text.splitlines()[0])

y = float(re.findall(r'\(.*?\)', driver.find_elements(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/div[2]')[0].text.splitlines()[27])[0][1:-2]) / 100

print("Stock Price:", S)
print("Dividend Yield:", y)

Stock Price: 138.0
Dividend Yield: 0.0003


Calculate 20 day annualized volatility

In [ ]:
# get historical data for volatility
historical = driver.find_element(By.XPATH, '//*[@id="nimbus-app"]/section/section/aside/section/nav/ul/li[5]/a')
# create action chain object
action = ActionChains(driver)
# click the item
action.click(on_element = historical)
# perform the operation
action.perform()

time.sleep(time_sleep)

#20 day average volatility
days = 20

historical_table = driver.find_elements(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/div[1]/div[3]/table')[0].text.splitlines()

#print("historical_table:", historical_table)

vol_row_log = []
Y = []

for i in range(days):
  adj_close = float((historical_table[i + 3].split())[-2])

  #print(adj_close)
  vol_row_log.append(log(adj_close))

#print("vol_row_log:", vol_row_log)

for i in range(days - 1):
  Y.append(vol_row_log[i] - vol_row_log[i+1])

#print("Y:", Y)

Y_avg = statistics.fmean(Y)
#print('Y_avg:',Y_avg)

vol = sqrt(sum((np.array(Y) - Y_avg)**2) / (days - 1)) * sqrt(252)
print("20 day volatility", vol)

20 day volatility 0.36816474112075825


In [ ]:
#go to options page
url = "https://sg.finance.yahoo.com/quote/"+ option + "/"

# Open the webpage
driver.get(url)

# Wait for the page to load
time.sleep(0.3)

option_table = driver.find_elements(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/div[2]')[0].text.splitlines()

K = float(option_table[9])
maturity = datetime.combine(datetime.strptime(option_table[11], '%Y-%m-%d').date(), datetime.strptime("11:59:00", '%H:%M:%S').time()) #add 11:59:00 to maturity date
print(option_table)

call_price = float(driver.find_elements(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/section[1]/div[2]/div[1]/section/div/section/div[1]/fin-streamer[1]')[0].text.splitlines()[0])

print("Strike_Price:", K)
print("call_price:", call_price)
print("Maturity:", maturity)

['Previous close', '13.74', 'Open', '14.50', 'Bid', '13.90', 'Ask', '14.00', 'Strike', '140.00', 'Expiry date', '2025-01-17', "Day's range", '13.65 - 14.60', 'Contract range', '--', 'Volume', '6,534', 'Open interest', '63.42k']
Strike_Price: 140.0
call_price: 13.97
Maturity: 2025-01-17 11:59:00


In [ ]:
# get_url = driver.current_url
# print("The current url is:"+str(get_url))

The current url is:https://sg.finance.yahoo.com/quote/NVDA241018C00020000/


In [ ]:
#current time
date = datetime.now(tz=pytz.timezone('America/New_York')).replace(microsecond = 0).replace(tzinfo=None)
print("NYC current date and time:", date)

time_to_maturity = maturity - date
print("Time to maturity:", time_to_maturity)
print(time_to_maturity.days, "days", time_to_maturity.seconds, "seconds")

#time to maturity (seconds)
tau_seconds = float(time_to_maturity.days) * 24 * 3600 + float(time_to_maturity.seconds)
print(tau_seconds, "seconds")

#time to maturity (years)
tau_years = tau_seconds / (365 * 24 * 3600)
print(tau_years, "years")

NYC current date and time: 2024-10-20 03:13:37
Time to maturity: 89 days, 8:45:23
89 days 31523 seconds
7721123.0 seconds
0.24483520421106036 years


In [ ]:
bsm_price = c(S, K, r, y, tau_years, vol)
print("Call_price:", call_price)
print("BSM_price:", bsm_price)

Call_price: 13.97
BSM_price: 9.164683873272118


In [ ]:
#number of shares to hold
delta = deltac(S, K, r, y, tau_years, vol)
print("Delta:", delta)

#value of shares
value_of_shares = delta * S
print("Value of shares:", value_of_shares)

#number of shares to buy
print("Buy", delta, "shares of stock")

#amount to borrow from bank
difference = delta * S - call_price
print("Long difference of", delta, "*", S, "-", call_price, "from bank =", -difference)

Delta: 0.5062912980168157
Value of shares: 69.86819912632058
Buy 0.5062912980168157 shares of stock
Long difference of 0.5062912980168157 * 138.0 - 13.97 from bank = -55.89819912632058


In [ ]:
#wealth
portfolio = delta * S - difference
print("Value of replicating portfolio:", portfolio)

#profit
profit = format(portfolio - call_price, '.10f')
print("Profit", profit)

Value of replicating portfolio: 13.969999999999999
Profit -0.0000000000


In [ ]:
#create dataframe
df = pd.DataFrame()

#create new portfolio entry
row = [date, maturity, S, K, difference, r, y, vol, bsm_price, call_price, delta, portfolio, profit]

df = pd.concat([df, pd.DataFrame([row])],axis=0, ignore_index=True)

df.columns = ['Date_Time', 'Maturity', 'Stock_Price', 'Strike_Price', 'Amount_Borrowed_from_Bank', 'r', 'Dividend_Yield', 'Vol', 'BSM_Price', 'Call_Price', 'Delta', 'Wealth', 'Profit']

df.head()

,Date_Time,Maturity,Stock_Price,Strike_Price,Amount_Borrowed_from_Bank,r,Dividend_Yield,Vol,BSM_Price,Call_Price,Delta,Wealth,Profit
0,2024-10-20 03:13:37,2025-01-17 11:59:00,138.0,140.0,55.898199,0.0034,0.0003,0.368165,9.164684,13.97,0.506291,13.97,-0.0000000000


In [ ]:
#export csv file
from google.colab import files
df.to_csv('NVDA.csv', date_format='%Y-%m-%d %H:%M:%S', index = False)
files.download("NVDA.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **To be ran every 5 minutes:**

In [ ]:
time_sleep = 0.8

#stock to search
option = 'NVDA250117C00140000'
ticker = 'NVDA'
r = 0.0034 #risk free sg rate as of oct 2024

In [ ]:
import pandas as pd
import time
import re
import os
import pyodbc
import pytz
import numpy as np
import statistics
from math import *
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

def web_driver():
  options = webdriver.ChromeOptions()
  options.add_argument("--verbose")
  options.add_argument("--no-sandbox")
  options.add_argument("--headless")
  options.add_argument("--disable-gpu")
  options.add_argument("--window-soze = 1920, 1200")
  options.add_argument("--disable-dev-shm-usage")
  driver = webdriver.Chrome(options = options)
  return driver

driver = web_driver()

def N(x):
  return (1.0 + erf(x / sqrt(2.0))) / 2.0

def deltac(S, K, r, y, τ, σ):
  d1 = 1/(σ * np.sqrt(τ)) * (np.log(S * np.exp(-y * (τ)) / K) + (r - y + σ**2/2)*(τ) )
  return np.exp(-y * τ) * N(d1)

def c(S, K, r, y, τ, σ):
  d1 = 1/(σ * np.sqrt(τ)) * (np.log(S * np.exp(-y * (τ)) / K) + (r - y + σ**2/2)*(τ) )
  d2 = d1 - σ * np.sqrt(τ)
  return S * np.exp(-y * (τ)) * N(d1) - K * np.exp(-r * (τ)) * N(d2)

In [ ]:
df = pd.read_csv(save_path + "NVDA.csv", date_format='%Y-%m-%d %H:%M:%S')
df.head()

,Date_Time,Maturity,Stock_Price,Strike_Price,Amount_Borrowed_from_Bank,r,Dividend_Yield,Vol,BSM_Price,Call_Price,Delta,Wealth,Profit
0,20/10/2024 2:48,17/1/2025 11:59,138,140,55.899116,0.0034,0.0003,0.368165,9.165654,13.97,0.506298,13.97,0


In [ ]:
#go to NVDA stock page
url = "https://sg.finance.yahoo.com/quote/"+ ticker + "/"

# Open the webpage
driver.get(url)

# Wait for the page to load
time.sleep(time_sleep)

S = float(driver.find_elements(By.XPATH, '/html/body/div[2]/main/section/section/section/article/section[1]/div[2]/div[1]/section/div/section[1]/div[1]/fin-streamer[1]')[0].text.splitlines()[0])

y = float(re.findall(r'\(.*?\)', driver.find_elements(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/div[2]')[0].text.splitlines()[27])[0][1:-2]) / 100

print("Stock Price:", S)
print("Dividend Yield:", y)

Stock Price: 138.0
Dividend Yield: 0.0003


In [ ]:
# get historical data for volatility
historical = driver.find_element(By.XPATH, '//*[@id="nimbus-app"]/section/section/aside/section/nav/ul/li[5]/a')
# create action chain object
action = ActionChains(driver)
# click the item
action.click(on_element = historical)
# perform the operation
action.perform()

time.sleep(time_sleep)

#20 day average volatility
days = 20

historical_table = driver.find_elements(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/div[1]/div[3]/table')[0].text.splitlines()

vol_row_log = []
Y = []

for i in range(days):
  adj_close = float((historical_table[i + 3].split())[-2])
  vol_row_log.append(log(adj_close))

for i in range(days - 1):
  Y.append(vol_row_log[i] - vol_row_log[i+1])

Y_avg = statistics.fmean(Y)

#get annual volatility
vol = sqrt(sum((np.array(Y) - Y_avg)**2) / (days - 1)) * sqrt(252)
print("20 day volatility:", vol)

20 day volatility: 0.36816474112075825


In [ ]:
#go to options page to get strike price and call price
url = "https://sg.finance.yahoo.com/quote/"+ option + "/"

# Open the webpage
driver.get(url)

# Wait for the page to load
time.sleep(time_sleep)

option_table = driver.find_elements(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/div[2]')[0].text.splitlines()

K = float(option_table[9])
print(option_table)

call_price = float(driver.find_elements(By.XPATH, '//*[@id="nimbus-app"]/section/section/section/article/section[1]/div[2]/div[1]/section/div/section/div[1]/fin-streamer[1]')[0].text.splitlines()[0])

print("Strike_Price:", K)
print("Call_Price:", call_price)

['Previous close', '13.74', 'Open', '14.50', 'Bid', '13.90', 'Ask', '14.00', 'Strike', '140.00', 'Expiry date', '2025-01-17', "Day's range", '13.65 - 14.60', 'Contract range', '--', 'Volume', '6,534', 'Open interest', '63.42k']
Strike_Price: 140.0
call_price: 13.97


In [ ]:
#current time
date = datetime.now(tz=pytz.timezone('America/New_York')).replace(microsecond = 0).replace(tzinfo=None)
print("NYC current date and time:", date)

#get time to maturity using 1st row
maturity = datetime.strptime(df['Maturity'].iloc[0], '%Y-%m-%d %H:%M:%S')
time_to_maturity = maturity - date
print("Time to maturity:", time_to_maturity)
print(time_to_maturity.days, "days", time_to_maturity.seconds, "seconds")

#time to maturity (seconds)
tau_seconds = float(time_to_maturity.days) * 24 * 3600 + float(time_to_maturity.seconds)
print(tau_seconds, "seconds")

#time to maturity (years)
tau_years = tau_seconds / (365 * 24 * 3600)
print(tau_years, "years")

NYC current date and time: 2024-10-20 02:57:51
Time to maturity: 89 days, 9:01:09
89 days 32469 seconds
7722069.0 seconds
0.24486520167427703 years


In [ ]:
bsm_price = c(S, K, r, y, tau_years, vol)
print("Call_Price:", call_price)
print("BSM_Price:", bsm_price)

Call_Price: 13.97
BSM_Price: 9.165303689185855


In [ ]:
delta = df['Delta'].iloc[-1]

K = df['Strike_Price'].iloc[0]

new_delta = deltac(S, K, r, y, tau_years, vol) #vols to be calculated daily

print("Current delta:", delta)
print("New delta:", new_delta)

print("Sell off", delta, "-", new_delta, "shares of stock")

change_in_delta = delta - new_delta
print("Change in delta:", change_in_delta)

Current delta: 0.5062979399650054
New delta: 0.5062955407523099
Sell off 0.5062979399650054 - 0.5062955407523099 shares of stock
Change in delta: 2.39921269551413e-06


In [ ]:
previous_date = datetime.strptime(df['Date_Time'].iloc[-1], '%Y-%m-%d %H:%M:%S')
print("Previous date:", previous_date)

elapsed_days = (date - previous_date).days
elapsed_seconds = (date - previous_date).seconds

elapsed_time_seconds = elapsed_days * 24 * 3600 + elapsed_seconds
print("Elapsed time (seconds):", elapsed_time_seconds)

elapsed_time_years = elapsed_time_seconds / (365 * 24 * 3600)
print("Elapsed time (years):", elapsed_time_years)

Previous date: 2024-10-20 02:48:56
Elapsed time (seconds): 535
Elapsed time (years): 1.6964738711314055e-05


In [ ]:
#profit from selling shares
share_profit = change_in_delta * S
print("Profit from shares:", share_profit)

#amount owed to bank before calculating interest
print("Current amount owed to bank before interest:", df['Amount_Borrowed_from_Bank'].iloc[-1])

#amount owed to bank after calculating interest
amount = df['Amount_Borrowed_from_Bank'].iloc[-1] * np.exp(r * elapsed_time_years)
print("Current amount owed to bank after interest:", amount)

#new amount owed to bank after repaying profit
new_amount = amount - share_profit
print("New amount owed to bank after repaying profit from change in shares:", new_amount)

Profit from shares: 0.00033109135198094997
Current amount owed to bank before interest: 55.89911571517075
Current amount owed to bank after interest: 55.89911893943808
New amount owed to bank after repaying profit from change in shares: 55.8987878480861


In [ ]:
#wealth
portfolio = delta * S - new_amount
print("Value of Replicating Portfolio:", portfolio)

#current profit
profit = format(portfolio - call_price, '.10f')
print("Profit from Closing Position:", profit)

Value of Replicating Portfolio: 13.970327867084649
Profit from Closing Position: 0.0003278671


In [ ]:
#add new row
row = [date, maturity, S, K, new_amount, r, y, vol, bsm_price, call_price, delta, portfolio, profit]

df.loc[len(df.index)] = row
df.head()

,Date_Time,Maturity,Stock_Price,Strike_Price,Amount_Borrowed_from_Bank,r,Dividend_Yield,Vol,BSM_Price,Call_Price,Delta,Wealth,Profit
0,20/10/2024 2:48,17/1/2025 11:59,138.0,140.0,55.899116,0.0034,0.0003,0.368165,9.165654,13.97,0.506298,13.970000,0
1,2024-10-20 02:57:51,2025-01-17 11:59:00,138.0,140.0,55.898788,0.0034,0.0003,0.368165,9.165304,13.97,0.506298,13.970328,0.0003278671


In [ ]:
df.head()

,Date_Time,Maturity,Stock_Price,Strike_Price,Amount_Borrowed_from_Bank,r,Dividend_Yield,Vol,BSM_Price,Call_Price,Delta,Wealth,Profit
0,20/10/2024 2:48,17/1/2025 11:59,138.0,140.0,55.899116,0.0034,0.0003,0.368165,9.165654,13.97,0.506298,13.970000,0
1,2024-10-20 02:57:51,2025-01-17 11:59:00,138.0,140.0,55.898788,0.0034,0.0003,0.368165,9.165304,13.97,0.506298,13.970328,0.0003278671


In [ ]:
from google.colab import files
df.to_csv('NVDA.csv', date_format='%Y-%m-%d %H:%M:%S', index = False)
files.download("NVDA.csv")

#delete last row
# df = df[:-1]
# df.head()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#export csv file

# from google.colab import files
# df.to_csv('NVDA.csv')
# files.download('NVDA.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Scripting**

In [ ]:
#stock to search
option = 'NVDA250117C00140000'
ticker = 'NVDA'
r = 0.0034 #risk free sg rate as of oct 2024

In [ ]:
#connect to Azure SQL server using pyodbc
server = '*****'
database = '*****'
username = '*****'
password = '*****'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

#delete data in table
cursor.execute('''TRUNCATE TABLE dbo.NVDA''')

#execute command
cnxn.commit()

In [ ]:
#export csv file
df.to_csv(r'/home/briansim74/Desktop/NVDA.csv', index = False)

In [ ]:
#BCP utility bulk insert updated CSV file into SQL server
os.system('/opt/mssql-tools18/bin/bcp dbo.NVDA in "/home/briansim74/Desktop/NVDA.csv" -S *****.database.windows.net -d ***** -U ***** -P ***** -c -F 2 -t "," -r "0x0a"')

In [ ]:
#close cursor
cursor.close()

In [ ]:
#quit driver
driver.quit()